[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/schwartz-cnl/Computational-Neuroscience-Class/blob/main/Convolutional%20Neural%20Network/Experiment_Visualize_pretrained_model.ipynb)

This notebook shows you how to load a pretrained keras model, and how to get intermediate layer activations. This is useful in the context of interpreting intermediate deep neural network representations, and for running simulations to compare deep neural network representations to biological visual cortex experiments. The notebook then shows an example of in-silico neurophysiology experiments of probing an artificial neuron with Gabor-like stimuli, and a simple version of visualizing a neuron feature properties.

Code by Xu Pan and Odelia Schwartz, with some code adapted from: https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1011486 The paper code: https://gin.g-node.org/xupan/CNN_surround_effects_visualization

In [ ]:
!wget https://github.com/schwartz-cnl/Computational-Neuroscience-Class/blob/main/Convolutional%20Neural%20Network/ILSVRC2012_test_00026783.JPEG?raw=true -O ILSVRC2012_test_00026783.JPEG

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow import keras
import tensorflow as tf
import numpy as np

# download the pretrained VGG16 model
model = VGG16(weights='imagenet', include_top=True)

In [ ]:
model.summary()

In [ ]:
# Let's test an image. Note the rest of this tutorial will use grating images
# load image
img_path = 'ILSVRC2012_test_00026783.JPEG'
# preprocess. We use the Keras preprocess_input function.
# We alway need to preprocess the input in the same way that the training prepreocessed the input.
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
# predict one-hot label
y = model.predict(x)

## Get intermediate layer activation

In [ ]:
# We can access an intermediate network layer. You could see the structure of VGG layers in
# the model.summary above
partial_model = keras.Model(inputs=model.input, outputs=model.get_layer('block3_conv3').output)

In [ ]:
response = partial_model.predict(x)

In [ ]:
response.shape
# For example, for block3_conv3, this gives us a 56x56 spatial map for each of 256 neurons

## In-silico electrophysiology
Let's try to explore some basic properties of the artificial neurons. A good starting point is to find the orientation and spatial period tuning curves.
This is commonly done, for instance, in neurophysiology experiments of
Primary Visual Cortex (V1) neurons.

In [ ]:
# @title A util function to generate stimuli (run this cell)

import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow

def makeGaussian(size, fwhm = 100, center=None):
    """
    Adapt from Andrew Aiessel.
    Make a square gaussian kernel.
    size is the length of a side of the square
    fwhm is full-width-half-maximum, which
    can be thought of as an effective radius.
    """

    x = np.arange(0, size, 1, float)
    y = x[:,np.newaxis]

    if center is None:
        x0 = y0 = size // 2
    else:
        x0 = center[0]
        y0 = center[1]

    return np.exp(-4*np.log(2) * ((x-x0)**2 + (y-y0)**2) / fwhm**2)

def makeGrating(size, spatialp, ori=0, phase=0, imsize=224, dtype='uint8'):
    """
    Make a square grating.
    size: the full-width-half-maximum of gaussian mask
    which can be thought of as an effective radius.
    spatialf: spatial frequency.
    ori: orientation, 0 is horizental. 90 is vertical.
    phase: 0-360
    imsize: the image size.
    """
    im = np.ones(imsize)
    # the last term is to make center phase 0.
    im = im*[np.sin(2*np.pi/spatialp*x+(phase/np.pi*180-2*np.pi/spatialp*imsize/2)) for x in range(imsize)]
    im = np.repeat(im[:,np.newaxis],imsize,axis=1)

    gaussianmask = makeGaussian(imsize, size)
    im = im*gaussianmask

    im = Image.fromarray(im)
    im = im.rotate(ori)

    im = (np.array(im)+1) / 2 * 255
    im = np.repeat(im[:,:,np.newaxis],3,axis=2)
    im = im.astype(dtype)
    return im

In [ ]:
# let see some examples of stimuli
import matplotlib.pyplot as plt

sti1 = makeGrating(size=50, spatialp=20, ori=0, phase=0, imsize=224)
plt.figure()
plt.imshow(sti1)

sti1 = makeGrating(size=50, spatialp=50, ori=45, phase=0, imsize=224)
plt.figure()
plt.imshow(sti1)

sti1 = makeGrating(size=50, spatialp=5, ori=90, phase=0, imsize=224)
plt.figure()
plt.imshow(sti1)

## We will do some simple in-silico neurophysiology on example model neurons. First, we run a tuning heatmap for a neuron

In [ ]:
def tuningHeatmap(oris, periods, partial_model, spatial_x, spatial_y, neuron):
  responses = np.zeros((len(oris), len(periods)))
  for i, ori in enumerate(oris):
      for j, period in enumerate(periods):
          phase_responses = [];
          for phase in range(0, 316, 45):  # average over responses to different phases, echoing biology experiments
               # Construct the stimulus with phase variation
               # Here we set the grating size to 40 to match the block 3, conv 3 theoretical RF size
               # Note website  about RF arithmetic:
               # https://blog.mlreview.com/a-guide-to-receptive-field-arithmetic-for-convolutional-neural-networks-e0f514068807
               sti = makeGrating(size=40, spatialp=period, ori=ori, phase=phase, imsize=224)
               sti = np.expand_dims(sti, axis=0)
               # The stimuli need to be processed in the same way model was trained!
               sti = preprocess_input(sti)
               # run the model
               prediction = partial_model.predict(sti, verbose=0)
               responses_onephase = prediction[0, spatial_x // 2, spatial_y // 2, neuron] # center of the feature map
               phase_responses.append(responses_onephase)
          # Average responses across different phases
          responses[i, j] = np.mean(phase_responses)

  # Find the indices of the maximum response
  max_index = np.argmax(responses)
  maxori, maxperiod = np.unravel_index(max_index, responses.shape)
  # plot the tuning heatmap
  plt.pcolor(oris, periods, responses.T)
  plt.colorbar()
  plt.xlabel("orientation")
  plt.ylabel("spatial period")
  # Return the indices corresponding to the maximum ori and freq
  return maxori, maxperiod, responses

In [ ]:
# Call the TuningHeatmap function, plot the heatmap and find the maximal orientation
# and spatial frequency indices
partial_model = keras.Model(inputs=model.input, outputs=model.get_layer('block3_conv3').output)
modelshape = partial_model.output_shape # tensorflow shape
print(modelshape)
spatial_x = modelshape[1]
spatial_y = modelshape[2]
print(spatial_x,spatial_y)
neuron = 5
oris = [x for x in range(0,180,15)]
periods = [x for x in range(5,35,5)]

[maxori_ind, maxperiod_ind, theresponses] = tuningHeatmap(oris, periods, partial_model, spatial_x, spatial_y, neuron)
maxori = oris[maxori_ind]
maxperiod = periods[maxperiod_ind]
print([maxori,maxperiod])

## Heatmap and size tuning curve for a neuron

In [ ]:
# In this paper, we created the tuning heatmap as above, and ran a range of in-silico experiments on artificial neural networks:
# https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1011486
# The paper code: https://gin.g-node.org/xupan/CNN_surround_effects_visualization
# Let's look at an example of increasing the diameter/size of a grating and testing the neural response
# (this is known as a diameter or size tuning curve)

# Here we go through some basic examples

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def compute_size_tuning_curve(freq, orient, spatial_x, spatial_y, neuron, partial_model, maxgratingSize):
   # Vary the grating sizes
   sizes = list(range(1, maxgratingSize, 2))
   responses = np.zeros(len(sizes))
   for i, size in enumerate(sizes):
        phase_responses = []
        for phase in range(0, 316, 45):
            sti = makeGrating(size=size, spatialp=freq, ori=orient, phase=phase, imsize=224)
            if np.mod(i,5)==0 and phase==0:
               plt.figure()
               plt.draw();
               plt.imshow(sti)
               plt.pause(0.1)  # Pause for visualization
            # Preprocess input
            sti = np.expand_dims(sti, axis=0)
            sti = preprocess_input(sti)
            # run the model
            prediction = partial_model.predict(sti, verbose=0)
            # Get neuron response
            response = prediction[0, spatial_x//2, spatial_y//2, neuron]
            phase_responses.append(response)
        # Compute the average response across the different phases
        responses[i] = np.mean(phase_responses)
   return sizes, responses

In [ ]:
# Plot the size tuning curve for a neuron

# Choose the neuron again from an intermediate layer
partial_model = keras.Model(inputs=model.input, outputs=model.get_layer('block3_conv3').output)
modelshape = partial_model.output_shape # tensorflow shape
print(modelshape)
spatial_x = modelshape[1]
spatial_y = modelshape[2]
print(spatial_x,spatial_y)
neuron = 5
print(neuron)

# This is the heatmap function from before
oris = [x for x in range(0,180,15)]
periods = [x for x in range(5,35,5)]
[maxori_ind, maxperiod_ind, theresponses] = tuningHeatmap(oris, periods, partial_model, spatial_x, spatial_y, neuron)
maxori = oris[maxori_ind]
maxperiod = periods[maxperiod_ind]
print([maxori,maxperiod])

# This is the size tuning
maxgratingSize = 80
sizes, responses = compute_size_tuning_curve(maxperiod, maxori, spatial_x, spatial_y, neuron, partial_model, maxgratingSize)
plt.figure()
plt.plot(sizes, responses, marker='o', linestyle='-')
plt.xlabel("Size")
plt.ylabel("Response")
plt.title("Size Tuning Curve")
plt.show()

In [ ]:
# Try another neuron

# Choose the neuron again from an intermediate layer
partial_model = keras.Model(inputs=model.input, outputs=model.get_layer('block3_conv3').output)
modelshape = partial_model.output_shape # tensorflow shape
print(modelshape)
spatial_x = modelshape[1]
spatial_y = modelshape[2]
print(spatial_x,spatial_y)
neuron = 2
print(neuron)

# This is the heatmap function from before
oris = [x for x in range(0,180,15)]
periods = [x for x in range(5,35,5)]
[maxori_ind, maxperiod_ind, theresponses] = tuningHeatmap(oris, periods, partial_model, spatial_x, spatial_y, neuron)
maxori = oris[maxori_ind]
maxperiod = periods[maxperiod_ind]
print([maxori,maxperiod])

# This is the size tuning
maxgratingSize = 80
sizes, responses = compute_size_tuning_curve(maxperiod, maxori, spatial_x, spatial_y, neuron, partial_model, maxgratingSize)
plt.figure()
plt.plot(sizes, responses, marker='o', linestyle='-')
plt.xlabel("Size")
plt.ylabel("Response")
plt.title("Size Tuning Curve")
plt.show()

## Feature visualization
Now let's visualize a neuron's feature.

In [ ]:
partial_model = keras.Model(inputs=model.input, outputs=model.get_layer('block3_conv3').output)
neuron = 5

opt = tf.keras.optimizers.Adam(learning_rate=0.01)

init_val = np.random.normal(size=(1,224,224,3), scale=0.01).astype(np.float32)
im = tf.Variable(init_val)
for epoch in range(100):
  with tf.GradientTape() as tape:
    loss = -tf.reduce_mean(partial_model(im)[0,:,:,neuron]) # here we are using the whole spatial map of the neuron
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, loss = {loss.numpy():.5f}")
  gradients = tape.gradient(loss, [im])
  opt.apply_gradients(zip(gradients, [im]))

plt.imshow(im[0,...])

In [ ]:
# let's try visualizing another neuron.

partial_model = keras.Model(inputs=model.input, outputs=model.get_layer('block4_conv2').output)
neuron = 2

opt = tf.keras.optimizers.Adam(learning_rate=0.01)

init_val = np.random.normal(size=(1,224,224,3), scale=0.01).astype(np.float32)
im = tf.Variable(init_val)
for epoch in range(100):
  with tf.GradientTape() as tape:
    loss = -tf.reduce_mean(partial_model(im)[0,:,:,neuron])
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, loss = {loss.numpy():.5f}")
  gradients = tape.gradient(loss, [im])
  opt.apply_gradients(zip(gradients, [im]))

plt.imshow(im[0,...])

### Note on the visualization of features
Above is only a minimal example of visualization. (An immediate but minor problem is we ignored the preprocess.) It often requires several tricks to obtain clear vivid features. An important trick is to use a natural image prior. This is usually done by putting a regulation loss on the Fourier spectrum (natural images are known to have spectrum slope of around 1.0).


This distill article is a good learning resource: https://distill.pub/2017/feature-visualization/

A newer study on more tricks: https://arxiv.org/pdf/2201.12961.pdf